# Data cleanup
This will outline the process that I am choosing along with the reasons why
I have chosen to further remove variables from my dataset. 

## Basic Plan
I have had some trouble with deciding as to whether or not I should attempt to replace the data the I had missing,
replace it with a value that indicates missing data (such as -1), or just remove the data. My thoughts on 
replacing the data went along the line of using the hourly data that I had to create minute data. Doing this by 
up-sampling the hourly data. After talking to Dr. Butler though I have instead decided to just drop the rows of missing
solar data, including rows where the quality control variable indicates a problem. While replacing all other columns
with missing or bad data with a -1 value. That way I only remove essential data rows where forecasting is not a 
possibility. While at the same time preserving the forecasting potential of the rest of the data.

## Entire Variable Removal With Reason
Based on year 2007 minute data I have to drop the following variables, reason included.
visible_cloud_optical_depth: 82.4% missing data
cloud_radiating_temperature: 33.2% missing data

All 6 years of training data variables removed

Variable name                    Percentage of data missing
pwd_cumul_rain                   0.701933
pwd_cumul_snow                   0.702052
pwd_err_code                     0.682465
pwd_mean_vis_10min               0.701172
pwd_mean_vis_1min                0.701133
pwd_precip_rate_mean_1min        0.701931
pwd_pw_code_15min                0.701926
pwd_pw_code_1hr                  0.701916
pwd_pw_code_inst                 0.701915
pws_cumul_rain                   0.361853
pws_cumul_snow                   0.361888
pws_err_code                     0.330972
pws_mean_vis_10min               0.361780
pws_mean_vis_1min                0.361782
qc_pwd_cumul_rain                0.670197
qc_pwd_cumul_snow                0.670197
qc_pwd_mean_vis_10min            0.670197
qc_pwd_mean_vis_1min             0.670197
qc_pwd_precip_rate_mean_1min     0.670197
qc_pwd_pw_code_15min             0.670197
qc_pwd_pw_code_1hr               0.670197
qc_pwd_pw_code_inst              0.670197
qc_pws_cumul_rain                0.329803
qc_pws_cumul_snow                0.329803
qc_pws_mean_vis_10min            0.329803
qc_pws_mean_vis_1min             0.329803

These variables are being dropped since all that I'm doing for the missing values is inputting a -1



## Additional Concerns
I believe that before I can remove the missing data rows from the data that I have I need to setup my sliding window.
The reason that I think this is that without setting up the sliding window with the correct forecasting time step
before I drop the missing or wrong data then I cannot ensure that I have the right forecasting placement. I also think
that when I do this I will need to do it to the Validation and Testing datasets at the same time. This way although I
don't use those datasets to train my models I have them in the correct format to use them correctly. So I will
do that then.


## Exploring the data

In [ ]:
import pandas as pd
import numpy as np
import os
# using 2007 as first example. Will check all years
df = pd.read_csv("/home/nelson/PycharmProjects/Solar Forecasting Thesis Project/"
                 "Data/train/2007/PreProcessed_data/minute_data_total_year.csv", index_col="time")
df.isnull().sum()  # sums up the missing data 

df.isnull().sum() / len(df)  # gives percentage of missing data by column

def find_null_values_with_continous_threshold(df, column, threshold):
    """
    Function useful for finding continous sections of missing data
    
    :param df: dataframe
    :param column: column name
    :param threshold: threshold of missing data to find
    :return: nothing
    """
    count = 0
    index_holder = 0
    i = 0
    while i < len(df.index) :
            if np.isnan(df[column].iloc[i]):
                for j in range(i, len(df.index)):
                    if count == 0:
                        index_holder = i
                    if not np.isnan(df[column].iloc[j]):
                        count = 0
                        index_holder = 0
                        break
                    count += 1
                    if count == threshold:
                        print("Starts at index " + df.index[index_holder])
                        print("10 missing at " + str(df.index[j]))
                    i += 1
                count = 0
                index_holder = 0
            i += 1
            

Here I will remove the columns that I have decided should no longer be in the dataset.

Add in forecasting column
Drop rows with missing solar data or bad quality bits

In [ ]:
# make this whole mess easier and put all training data into one file 
direct = os.listdir("Data/train")
direct.sort()

df = []
for item in direct:
    d = pd.read_csv("Data/train/" + item + "/PreProcessed_data/minute_data_total_year.csv", index_col="time")
    df.append(d)
    
df = pd.concat(df)

df = df.drop(columns=["visible_cloud_optical_depth", "cloud_radiating_temperature",
                      "pwd_cumul_rain", "pwd_cumul_snow",
                      "pwd_err_code", "pwd_mean_vis_10min",
                      "pwd_mean_vis_1min", "pwd_precip_rate_mean_1min",
                      "pwd_pw_code_15min", "pwd_pw_code_1hr",
                      "pwd_pw_code_inst", "pws_cumul_rain",
                      "pws_cumul_snow", "pws_err_code",
                      "pws_mean_vis_10min", "pws_mean_vis_1min",
                      "qc_pwd_cumul_rain", "qc_pwd_cumul_snow",
                      "qc_pwd_mean_vis_10min", "qc_pwd_mean_vis_1min",
                      "qc_pwd_precip_rate_mean_1min", "qc_pwd_pw_code_15min",
                      "qc_pwd_pw_code_1hr", "qc_pwd_pw_code_inst",
                      "qc_pws_cumul_rain", "qc_pws_cumul_snow",
                      "qc_pws_mean_vis_10min", "qc_pws_mean_vis_1min",
                      "qc_time"])  # dropping vars
df["forecast_downwelling_shortwave"] = df["downwelling_shortwave"].shift(20, axis=0)  # shifted values for forecast
df["qc_forecast_downwelling_shortwave"] = df["qc_downwelling_shortwave"].shift(20, axis=0)
df = df[df["qc_downwelling_shortwave"] == 0]  # removes 7% of the data. 
df = df[df["qc_forecast_downwelling_shortwave"] == 0] 


Out of the Training data that I have, by removing all of the rows that had a bad quality checker 
for either the downwelling shortwave or the forecasting downwelling shortwave removed 8.9% of the data.

Now I have to remove all other quality checker variables and replace nan values with a -1
I am also removing the shortwave quality checker variables.

In [ ]:
df = df.drop(columns=['qc_air_temperature', 'qc_dew_point_mean', 'qc_downwelling_shortwave',
       'qc_precipitation', 'qc_pressure', 'qc_relative_humidity',
       'qc_wind_direction', 'qc_wind_speed', 'qc_forecast_downwelling_shortwave',
                      'source_downwelling_shortwave'])

df.loc[df["downwelling_shortwave"] < 0, "downwelling_shortwave"] = 0  # cast less than 0 values as 0
df.loc[df["forecast_downwelling_shortwave"] < 0, "forecast_downwelling_shortwave"] = 0

Null variables by sum. Replacing all values greater that 0 with -1
air_temperature                    16020
brightness_temperature             15480
dew_point_mean                     15812
dew_point_std                          0
downwelling_shortwave                  0
precipitation                     211229
pressure                            7468
relative_humidity                  18798
source_downwelling_shortwave           0
vapor_pressure                     18883
wind_direction                         0
wind_speed                            32
forecast_downwelling_shortwave         0



In [ ]:
# Start replacing
df.loc[df["air_temperature"].isnull(), "air_temperature"] = -1
df.loc[df["brightness_temperature"].isnull(), "brightness_temperature"] = -1
df.loc[df["dew_point_mean"].isnull(), "dew_point_mean"] = -1
df.loc[df["precipitation"].isnull(), "precipitation"] = -1
df.loc[df["pressure"].isnull(), "pressure"] = -1
df.loc[df["relative_humidity"].isnull(), "relative_humidity"] = -1
df.loc[df["vapor_pressure"].isnull(), "vapor_pressure"] = -1
df.loc[df["wind_speed"].isnull(), "wind_speed"] = -1

#output to csv 
df.to_csv("Data/train/fully processing minute data.csv")

Further editing needs to to be done to this document mainly to document the change in data for both the testing
and validation datasets. Also editing but I think I've now got everything ready to go to run my first 
forecasting method.

train data shape changes
3680345, 49
3680345, 20
3403866, 22
3352696, 12


Running the same process on testing data at this time in the console. Using this section to keep
notes 

testing data shape changes
(1049520, 40)
(1043739, 23)
(1039841, 23)
(1039841, 12)

Ok so apparently my testing dataset has a different variables set than my training data
So I'm just going to keep all of this in final processing step in the notebook

In [ ]:
# Start processing test data

direct = os.listdir("Data/test")
direct.sort()

df = []
for item in direct:
    d = pd.read_csv("Data/test/" + item + "/PreProcessed_data/minute_data_total_year.csv", index_col="time")
    df.append(d)
df = pd.concat(df)

df.shape
df = df.drop(columns=["visible_cloud_optical_depth", "cloud_radiating_temperature",
                      "pwd_cumul_rain", "pwd_cumul_snow",
                      "pwd_err_code", "pwd_mean_vis_10min",
                      "pwd_mean_vis_1min", "pwd_precip_rate_mean_1min",
                      "pwd_pw_code_15min", "pwd_pw_code_1hr",
                      "pwd_pw_code_inst", 
                      "qc_pwd_cumul_rain", "qc_pwd_cumul_snow",
                      "qc_pwd_mean_vis_10min", "qc_pwd_mean_vis_1min",
                      "qc_pwd_precip_rate_mean_1min", "qc_pwd_pw_code_15min",
                      "qc_pwd_pw_code_1hr", "qc_pwd_pw_code_inst"])  # dropping vars

df["forecast_downwelling_shortwave"] = df["downwelling_shortwave"].shift(20, axis=0)  # shifted values for forecast
df["qc_forecast_downwelling_shortwave"] = df["qc_downwelling_shortwave"].shift(20, axis=0)
df = df[df["qc_downwelling_shortwave"] == 0]  # removes 0.5% of the data. 
df = df[df["qc_forecast_downwelling_shortwave"] == 0]

df = df.drop(columns=['source_downwelling_shortwave',
       'qc_downwelling_shortwave', 'qc_wind_speed',
                      'qc_wind_direction', 'qc_air_temperature', 'qc_relative_humidity',
                      'qc_pressure', 'qc_precipitation', 'qc_dew_point_mean',
                      'qc_trh_err_code', 'qc_forecast_downwelling_shortwave'])

df.loc[df["downwelling_shortwave"] < 0, "downwelling_shortwave"] = 0  # cast less than 0 values as 0
df.loc[df["forecast_downwelling_shortwave"] < 0, "forecast_downwelling_shortwave"] = 0

df.loc[df["air_temperature"].isnull(), "air_temperature"] = -1
df.loc[df["brightness_temperature"].isnull(), "brightness_temperature"] = -1
df.loc[df["dew_point_mean"].isnull(), "dew_point_mean"] = -1
df.loc[df["precipitation"].isnull(), "precipitation"] = -1
df.loc[df["pressure"].isnull(), "pressure"] = -1
df.loc[df["relative_humidity"].isnull(), "relative_humidity"] = -1
df.loc[df["vapor_pressure"].isnull(), "vapor_pressure"] = -1
df.loc[df["wind_speed"].isnull(), "wind_speed"] = -1
df.loc[df["wind_direction"].isnull(), "wind_direction"] = -1

df.to_csv("Data/test/fully processes minute data.csv")

Moving and finishing up with Validation data


Validation data shape changes

In [ ]:
direct = os.listdir("Data/validate")
direct.sort()
df = []
for item in direct:
    d = pd.read_csv("Data/validate/" + item + "/PreProcessed_data/minute_data_total_year.csv", index_col="time")
    df.append(d)
df = pd.concat(df)
df.shape

df = df.drop(columns=["visible_cloud_optical_depth", "cloud_radiating_temperature",
                      "pwd_cumul_rain", "pwd_cumul_snow",
                      "pwd_err_code", "pwd_mean_vis_10min",
                      "pwd_mean_vis_1min", "pwd_precip_rate_mean_1min",
                      "pwd_pw_code_15min", "pwd_pw_code_1hr",
                      "pwd_pw_code_inst", 
                      "qc_pwd_cumul_rain", "qc_pwd_cumul_snow",
                      "qc_pwd_mean_vis_10min", "qc_pwd_mean_vis_1min",
                      "qc_pwd_precip_rate_mean_1min", "qc_pwd_pw_code_15min",
                      "qc_pwd_pw_code_1hr", "qc_pwd_pw_code_inst"])  # dropping vars

df["forecast_downwelling_shortwave"] = df["downwelling_shortwave"].shift(20, axis=0)  # shifted values for forecast
df["qc_forecast_downwelling_shortwave"] = df["qc_downwelling_shortwave"].shift(20, axis=0)
df = df[df["qc_downwelling_shortwave"] == 0]  # removes 10.8% of the data. 
df = df[df["qc_forecast_downwelling_shortwave"] == 0]

df = df.drop(columns=['source_downwelling_shortwave',
       'qc_downwelling_shortwave', 'qc_wind_speed',
                      'qc_wind_direction', 'qc_air_temperature', 'qc_relative_humidity',
                      'qc_pressure', 'qc_precipitation', 'qc_dew_point_mean',
                      'qc_trh_err_code', 'qc_forecast_downwelling_shortwave', "qc_time"])

df.loc[df["downwelling_shortwave"] < 0, "downwelling_shortwave"] = 0  # cast less than 0 values as 0
df.loc[df["forecast_downwelling_shortwave"] < 0, "forecast_downwelling_shortwave"] = 0

df.loc[df["air_temperature"].isnull(), "air_temperature"] = -1
df.loc[df["brightness_temperature"].isnull(), "brightness_temperature"] = -1
df.loc[df["dew_point_mean"].isnull(), "dew_point_mean"] = -1
df.loc[df["precipitation"].isnull(), "precipitation"] = -1
df.loc[df["pressure"].isnull(), "pressure"] = -1
df.loc[df["relative_humidity"].isnull(), "relative_humidity"] = -1
df.loc[df["vapor_pressure"].isnull(), "vapor_pressure"] = -1
df.loc[df["wind_speed"].isnull(), "wind_speed"] = -1
df.loc[df["wind_direction"].isnull(), "wind_direction"] = -1

df.to_csv("Data/validate/fully processes minute data.csv")